In [31]:
from sklearn.datasets import load_boston
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [32]:
boston_data = load_boston()

In [33]:
boston_data

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [34]:
X_val = boston_data['data']
y_val = boston_data['target']

In [35]:
model = LinearRegression()

In [36]:
model.fit(X_val,y_val)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [37]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

In [38]:
rmse

4.679191295697282

In [39]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn import svm
import seaborn as sns;sns.set()

In [40]:
df = pd.read_csv('./algoTrade-master/ch06/ETFs_main.csv')
df.head()

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.20,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.07,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.42,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.79,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.95,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960


In [41]:
def moving_average(df,n):
    MA = pd.Series(df['CLOSE_SPY'].rolling(n,min_periods=n).mean(),name='MA_'+str(n))
    df = df.join(MA)
    return df

def volume_moving_average(df,n):
    VMA = pd.Series(df['VOLUME'].rolling(n,min_periods=n).mean(),name='VMA_'+str(n))
    df = df.join(VMA)
    return df

def relative_strength_index(df,n):
    i=0
    UpI=[0]
    DoI=[0]
    while i+1 <= df.index[-1]:
        UpMove = df.loc[i+1,'HIGH'] - df.loc[i,'HIGH']
        DoMove = df.loc[i,'LOW'] - df.loc[i+1,'LOW']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i+=1
        
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n,min_periods=n).mean())
    NegDI = pd.Series(DoI.ewm(span=n,min_periods=n).mean())
    RSI = pd.Series(PosDI / (PosDI+NegDI), name='RSI_'+str(n))
    df = df.join(RSI)
    return df

In [42]:
df = moving_average(df,45)
df = volume_moving_average(df,45)
df = relative_strength_index(df,14)

In [43]:
df.tail()

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
2766,2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.18,269.767778,1.240592e+08,0.172439
2767,2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.87,269.018889,1.274610e+08,0.142578
2768,2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.32,267.995333,1.281067e+08,0.115003
2769,2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.90,267.275778,1.297876e+08,0.307099
2770,2018-12-28,247.75,249.58,251.400,246.0,153100188.0,121.06,86.63,1.7320,0.45,9.53,25.50,28.34,38.17,266.639111,1.301996e+08,0.372653


In [44]:
df = df.set_index('Dates')
df = df.dropna()
len(df)

2727

In [45]:
df['target'] = df['CLOSE_SPY'].pct_change()

In [46]:
df.tail()

,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14,target
Dates,,,,,,,,,,,,,,,,,
2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.18,269.767778,1.240592e+08,0.172439,-0.016278
2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.87,269.018889,1.274610e+08,0.142578,-0.026176
2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.32,267.995333,1.281067e+08,0.115003,-0.026423
2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.90,267.275778,1.297876e+08,0.307099,0.058590
2018-12-28,247.75,249.58,251.400,246.0,153100188.0,121.06,86.63,1.7320,0.45,9.53,25.50,28.34,38.17,266.639111,1.301996e+08,0.372653,-0.001290


In [47]:
df['target'] = np.where(df['target']>0,1,-1)
df['target'].value_counts()

 1    1471
-1    1256
Name: target, dtype: int64

In [48]:
# 주가 트렌드 예측에서는 불균형 데이터로 간주
# 일반적으로 머신러닝 기반의 주가트렌드 분류모델은 54~56% 정도의 정확도를 달성하기 힘들다.
# 이 데이터를 기반으로 학습하면 오른다고만 찍어도 아래와 같은 정확도를 가지므로 정확도가 아닌 학습 평가 지표를 사용해야한다.
1471/(1471+1256)

0.5394206087275394

In [49]:
df['target'] = df['target'].shift(-1)

In [50]:
df = df.dropna()
len(df)

2726

In [51]:
df['target'] = df['target'].astype(np.int64)
y_var = df['target']
X_var = df.drop(['target','OPEN','HIGH','LOW','VOLUME','CLOSE_SPY'],axis=1)

In [52]:
X_var.head()

,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
Dates,,,,,,,,,,,
2007-04-27,67.56,83.7300,2.4474,0.55,51.84,24.54,12.45,41.750,143.551556,1.106696e+08,0.670018
2007-04-30,67.09,83.7166,2.4361,0.57,51.24,24.49,14.22,40.935,143.601556,1.116466e+08,0.531751
2007-05-02,66.66,83.3800,2.4366,0.59,49.59,24.66,13.08,42.020,143.680667,1.121613e+08,0.554050
2007-05-03,67.49,83.1100,2.4346,0.60,49.28,24.69,13.09,42.435,143.780222,1.123421e+08,0.601028
2007-05-04,68.19,83.2300,2.4006,0.60,48.30,24.60,12.91,42.595,143.905111,1.128853e+08,0.665987


In [53]:
up = df[df['target']==1].target.count()
total = df.target.count()
print('up/total ratio:{0:.2f}%'.format(up/total*100))

up/total ratio:53.96%


In [54]:
# 주가 데이터는 시계열이기 때문에 순서를 바꾸지 말아야한다.
X_train,X_test,y_train,y_test = train_test_split(X_var,y_var,test_size=0.3,shuffle=False,random_state=42)

In [55]:
train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts()/train_count)
print('\ntest set label ratio')
print(y_test.value_counts()/test_count)

train set label ratio
 1    0.543501
-1    0.456499
Name: target, dtype: float64

test set label ratio
 1    0.530562
-1    0.469438
Name: target, dtype: float64


In [56]:
def get_confusion_matrix(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    roc_score = roc_auc_score(y_test,pred)
    print('confusion matrix')
    print('accuracy:{0:.24f}, precision:{1:.4f}, recall:{2:.4f}, f1:{3:.4f}, ROC AUC_score:{3:.4f}'.format(accuracy,precision,recall,f1,roc_score))

In [59]:
from sklearn.metrics import accuracy_score
# 오른다고만 찍어도 54%의 정확도를 보이는데 오히려 정확도가 더 낮음
xgb_dis = XGBClassifier(n_estimators=400,learning_rate=0.1,max_depth=3)
xgb_dis.fit(X_train,y_train)
xgb_pred = xgb_dis.predict(X_test)
print(xgb_dis.score(X_train,y_train))
get_confusion_matrix(y_test,xgb_pred)

0.8763102725366876
confusion matrix
accuracy:0.491442542787286051897411, precision:0.5242, recall:0.4493, f1:0.4839, ROC AUC_score:0.4839


In [60]:
n_estimators=range(10,200,10)
params = {
    'bootstrap' : [True],
    'n_estimators' : n_estimators,
    'max_depth' : [4,6,8,10,12],
    'min_samples_leaf' : [2,3,4,5],
    'min_samples_split' : [2,4,6,8,10],
    'max_features':[4]
}

In [61]:
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)
my_cv

<generator object TimeSeriesSplit.split at 0x0000022B46D3ACC8>

In [63]:
from sklearn.ensemble import RandomForestClassifier
clf = GridSearchCV(RandomForestClassifier(),params,cv=my_cv,n_jobs=-1)
clf.fit(X_train,y_train)

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x0000022B46D3ACC8>,
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              m...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
  

In [64]:
clf.best_params_

{'bootstrap': True,
 'max_depth': 4,
 'max_features': 4,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 110}

In [65]:
clf.best_score_

0.5534591194968553

In [66]:
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.5134474327628362

In [67]:
get_confusion_matrix(y_test,y_pred)

confusion matrix
accuracy:0.513447432762836220021541, precision:0.5479, recall:0.4747, f1:0.5086, ROC AUC_score:0.5086


In [76]:
df = pd.read_csv('./algoTrade-master/ch06/ETFs_main.csv')
df = moving_average(df,45)
df = volume_moving_average(df,45)
df = relative_strength_index(df,14)
df = df.set_index('Dates')
df = df.dropna()

In [77]:
df['target'] = df['CLOSE_SPY'].pct_change()
df['target'].describe()

count    2726.000000
mean        0.000270
std         0.013026
min        -0.098448
25%        -0.004320
50%         0.000545
75%         0.005791
max         0.128249
Name: target, dtype: float64

In [78]:
df['target'] = np.where(df['target']>0.000545,1,-1)
df['target'].value_counts()

-1    1364
 1    1363
Name: target, dtype: int64

In [79]:
df['target'] = df['target'].shift(-1)
df = df.dropna()
print(len(df))

2726


In [80]:
df['target'] = df['target'].astype(np.int64)
y_var = df['target']
X_var = df.drop(['target','OPEN','HIGH','LOW','VOLUME','CLOSE_SPY'],axis=1)

In [81]:
up = df[df['target']==1].target.count()
total = df.target.count()
print('up/total ratio:{0:.2f}%'.format(up/total*100))

up/total ratio:50.00%


In [82]:
X_train,X_test,y_train,y_test = train_test_split(X_var,y_var,test_size=0.3,shuffle=False,random_state=42)

In [85]:
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)
clf = GridSearchCV(RandomForestClassifier(),params,cv=my_cv,n_jobs=-1)
clf.fit(X_train,y_train)

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x0000022B47FC68C8>,
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              m...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
  

In [87]:
clf.best_params_

{'bootstrap': True,
 'max_depth': 6,
 'max_features': 4,
 'min_samples_leaf': 4,
 'min_samples_split': 6,
 'n_estimators': 60}

In [88]:
clf.best_score_

0.5333333333333333

In [89]:
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.5183374083129584

In [91]:
get_confusion_matrix(y_test,y_pred)

confusion matrix
accuracy:0.518337408312958380918190, precision:0.5091, recall:0.1414, f1:0.2213, ROC AUC_score:0.2213


In [92]:
xgb_dis.fit(X_train,y_train)
xgb_pred = xgb_dis.predict(X_test)
print(xgb_dis.score(X_train,y_train))
get_confusion_matrix(y_test,xgb_pred)

0.8658280922431866
confusion matrix
accuracy:0.497555012224938864040524, precision:0.4801, recall:0.4571, f1:0.4683, ROC AUC_score:0.4683
